In [1]:
import pandas as pd
import numpy as np
import re
import ast

In [2]:
df = pd.read_csv("datos_areas_internet/areas_ac.csv")

In [3]:
df.head(2)

,WKT,nombre,descripción,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuito,Agua:si,vaciado:si,WC:si,electric,no),NaN,NaN
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuito,Agua:Si:Wc:en verano),NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def sacar_latitud(col):
    patron = "-?\d+\.\d+" 
    return re.findall(patron, col)[1]

def sacar_longitud(col):
    patron = "-?\d+\.\d+" 
    return re.findall(patron, col)[0]

In [5]:
df["latitud"]= df["WKT"].apply(sacar_latitud)

In [6]:
df["longitud"] = df["WKT"].apply(sacar_longitud)

In [7]:
df.head()

,WKT,nombre,descripción,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,latitud,longitud
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuito,Agua:si,vaciado:si,WC:si,electric,no),NaN,NaN,39.06956,-0.4098
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuito,Agua:Si:Wc:en verano),NaN,NaN,NaN,NaN,NaN,NaN,42.64159,-8.89696
2,POINT (0.35921 42.90715 0.0),Arreau Ã¡rea AC municipal (2 euros,Agua:si,vaciado:si,WC:no,electric,no),NaN,NaN,42.90715,0.35921
3,POINT (-0.41007 38.39479 0.0),Camper Area Campello Beach (14eur/noche,Agua:si,WC:si,Electr.:si 3eur,Ducha:si),NaN,NaN,NaN,38.39479,-0.41007
4,POINT (-8.44443 43.37179 0.0),A CoruÃ±a Gratuito: Si,Pernocta: Si,WC: Si,Agua: Si,NaN,NaN,NaN,NaN,43.37179,-8.44443


In [8]:
def sacar_precio(col):
    patron = "[G|g]ratuito|\d+" 
    try:
        return re.findall(patron, col)[0]
    except:
        return re.findall(patron, col)

In [9]:
df["precio"] = df["nombre"].apply(sacar_precio)

In [10]:
df.head()

,WKT,nombre,descripción,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,latitud,longitud,precio
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuito,Agua:si,vaciado:si,WC:si,electric,no),NaN,NaN,39.06956,-0.4098,Gratuito
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuito,Agua:Si:Wc:en verano),NaN,NaN,NaN,NaN,NaN,NaN,42.64159,-8.89696,Gratuito
2,POINT (0.35921 42.90715 0.0),Arreau Ã¡rea AC municipal (2 euros,Agua:si,vaciado:si,WC:no,electric,no),NaN,NaN,42.90715,0.35921,2
3,POINT (-0.41007 38.39479 0.0),Camper Area Campello Beach (14eur/noche,Agua:si,WC:si,Electr.:si 3eur,Ducha:si),NaN,NaN,NaN,38.39479,-0.41007,14
4,POINT (-8.44443 43.37179 0.0),A CoruÃ±a Gratuito: Si,Pernocta: Si,WC: Si,Agua: Si,NaN,NaN,NaN,NaN,43.37179,-8.44443,Gratuito


In [11]:
lista = ["Gratuito", "10", "7", "8", "5", "6", "12", "11", "4", "3", "15", "9", "2", "gratuito", "18", "20", "25", "13", "16", "26", "30", "19", "14", "300", "0", "340", "21", "28", "110", "17", "22", "124"]

In [12]:
df["precio"].value_counts()

Gratuito    581
10           55
7            33
[]           31
8            30
5            26
6            24
12           18
11           13
4            13
3            11
15           11
9            10
gratuito      7
2             7
18            6
20            6
25            5
13            5
16            4
26            2
30            2
19            2
14            2
300           1
110           1
17            1
22            1
124           1
0             1
340           1
21            1
28            1
Name: precio, dtype: int64

In [13]:
df[df["precio"] == "300"] # 300 SEK --> 26€

,WKT,nombre,descripción,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,latitud,longitud,precio
350,POINT (18.03195 59.32021 0.0),Area AC Langholmens Estocolmo (300 SEK,Agua:si,vaciado:si,WC:si,electric,si,duchas:si 5 SEK),NaN,59.32021,18.03195,300


In [14]:
df[df["precio"] == "340"] # --> 10 €

,WKT,nombre,descripción,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,latitud,longitud,precio
45,POINT (1.45677 41.16679 0.0),Area AC 340 (Creixell) (10 euros,Agua:si,vaciado:si,WC:no,electric:si,wifi:si,ducha:no),NaN,41.16679,1.45677,340


In [15]:
df2 = df[~df["precio"].isin(lista)]

In [16]:
df2.precio.value_counts()

[]    31
Name: precio, dtype: int64

In [17]:
df2.head(31)

,WKT,nombre,descripción,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,latitud,longitud,precio
24,POINT (-2.33781 43.03665 0.0),Ac Mirandaola. Legazpi Agua: Si,Wc: Si,Gratuito: Si,NaN,NaN,NaN,NaN,NaN,43.03665,-2.33781,[]
34,POINT (5.7117 43.68666 0.0),AC Sant Paul Les Durance Agua: Si,WC: No,Gratuito: Si,NaN,NaN,NaN,NaN,NaN,43.68666,5.7117,[]
40,POINT (13.09759 54.31272 0.0),Am Langenwall Agua: No,Wc: Si,Luz: No,Gratuito: No 4eur dia,NaN,NaN,NaN,NaN,54.31272,13.09759,[]
130,POINT (0.64585 45.3791 0.0),Area AC Brantome,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.3791,0.64585,[]
145,POINT (14.51606 46.09956 0.0),Area AC Campark Aleksander Kovac,LJUBLJANA (14eur/noche,Agua:si,WC:si,Electr.:si,Ducha:si),NaN,NaN,46.09956,14.51606,[]
163,POINT (0.07896 44.31055 0.0),Area AC Casteljaloux,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.31055,0.07896,[]
199,POINT (-4.39832 43.23353 0.0),Area Ac Cosio (Valle de Nansa) (Area privada,gratuito,agua,wc,vaciado),NaN,NaN,NaN,43.23353,-4.39832,[]
211,POINT (5.60792 48.24051 0.0),Area Ac de Goncourt Gratuita,Agua: si (2 eur),Wc: no,NaN,NaN,NaN,NaN,NaN,48.24051,5.60792,[]
241,POINT (-6.09289 42.9565 0.0),Area AC El Moriscal,Huergas de Babia (5 eur.,Agua:si,vaciado:si,WC:no,electric,no),NaN,42.9565,-6.09289,[]
242,POINT (-6.22103 36.59877 0.0),Area Ac El Puerto de Sta Maria (Area privada,6eur/noche,Agua:no,Wc:no,Vaciado:Si,Elec:no),NaN,NaN,36.59877,-6.22103,[]


In [18]:
df2["precio"] = df2["precio"].astype(str)

/tmp/ipykernel_570/180257307.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["precio"] = df2["precio"].astype(str)


In [19]:
df2["precio"].dtype

dtype('O')

In [20]:
df2["precio"] = df2["precio"].replace("[]", "Gratuito")

/tmp/ipykernel_570/3506447228.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["precio"] = df2["precio"].replace("[]", "Gratuito")


In [21]:
df2.head(1)

,WKT,nombre,descripción,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,latitud,longitud,precio
24,POINT (-2.33781 43.03665 0.0),Ac Mirandaola. Legazpi Agua: Si,Wc: Si,Gratuito: Si,NaN,NaN,NaN,NaN,NaN,43.03665,-2.33781,Gratuito


In [25]:
list_index = df2.index.tolist()

In [29]:
len(list_index)

31

In [26]:
df.drop(list_index, axis= 0, inplace= True)

In [30]:
df.shape

(882, 12)

In [31]:
df2.shape

(31, 12)

In [27]:
df_final = pd.concat([df, df2], axis= 0, ignore_index= True)
df_final.head()

,WKT,nombre,descripción,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,latitud,longitud,precio
0,POINT (-0.4098 39.06956 0.0),Area AC Hort de Soriano (Carcaixent) (Gratuito,Agua:si,vaciado:si,WC:si,electric,no),NaN,NaN,39.06956,-0.4098,Gratuito
1,POINT (-8.89696 42.64159 0.0),Area de ac playa de BarraÃ±a (Boiro) (Gratuito,Agua:Si:Wc:en verano),NaN,NaN,NaN,NaN,NaN,NaN,42.64159,-8.89696,Gratuito
2,POINT (0.35921 42.90715 0.0),Arreau Ã¡rea AC municipal (2 euros,Agua:si,vaciado:si,WC:no,electric,no),NaN,NaN,42.90715,0.35921,2
3,POINT (-0.41007 38.39479 0.0),Camper Area Campello Beach (14eur/noche,Agua:si,WC:si,Electr.:si 3eur,Ducha:si),NaN,NaN,NaN,38.39479,-0.41007,14
4,POINT (-8.44443 43.37179 0.0),A CoruÃ±a Gratuito: Si,Pernocta: Si,WC: Si,Agua: Si,NaN,NaN,NaN,NaN,43.37179,-8.44443,Gratuito


In [32]:
df_final.shape

(913, 12)

In [34]:
df_final[df_final["precio"] == "110"] # --> 10 €

,WKT,nombre,descripción,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,latitud,longitud,precio
16,POINT (13.65136 68.30725 0.0),AC Eggum (110coronas/noche,Agua:si,Wc:no),NaN,NaN,NaN,NaN,NaN,68.30725,13.65136,110


In [24]:
df.isnull().sum()

WKT              0
nombre           0
descripción     38
Unnamed: 3      55
Unnamed: 4     200
Unnamed: 5     300
Unnamed: 6     653
Unnamed: 7     869
Unnamed: 8     904
latitud          0
longitud         0
precio           0
dtype: int64

Get the location from coordinates:
https://openweathermap.org/api/geocoding-api